# INTRO
- here we try to combine the dictionary extraction task + visualisation with jeroen's new data
- we extract the dictionary of possible timestamp configurations and we render them. this should be something that goes on forever, so to enrich an existing dictionary etc

# IMPORTS

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import imageio

In [ ]:
from imports.general_settings import *
from imports.general_functions import *

In [ ]:
%load_ext autoreload
%autoreload 2

# RENDERING FUNCTIONS

In [ ]:
def make_EGO_IMG_cell(i, ego_lateral, ego_longitudinal, debug=False) :
    ego_img = np.zeros((EGO_IMG_H, EGO_IMG_W, 3))
    has_issue_longitudinal = False
    has_issue_lateral = False
    
    if debug :
        print(i, ego_lateral, ego_longitudinal)
        
    # processing longitutinal information
    if ego_longitudinal == 'a' :
        # accelerate
        ego_img += EGO_ACCELERATING_COLOUR
    elif ego_longitudinal == 'd' :
        ego_img += EGO_DECELERATING_COLOUR
    elif ego_longitudinal == 'c' :
        ego_img += EGO_CRUISING_COLOUR
    else :
        ego_img += EGO_ISSUE
        has_issue_longitudinal = True
        
    # processing lateral information
    if ego_lateral == 'r' :
        # we place the blinker at the front right
        x0 = EGO_IMG_W - BLINKER_W
        y0 = EGO_IMG_H - BLINKER_H
        ego_img[y0:y0+BLINKER_H, x0:x0+BLINKER_W] = EGO_BLINKER_COLOUR
    elif ego_lateral == 'l' :
        # we place the blinker at the front left
        x0 = EGO_IMG_W - BLINKER_W
        y0 = 0
        ego_img[y0:y0+BLINKER_H, x0:x0+BLINKER_W] = EGO_BLINKER_COLOUR
    elif ego_lateral == 'fl' :
        # lane following => no blinker
        pass
    else :
        ego_img += EGO_ISSUE
        has_issue_lateral = True
        
    cell_img = np.ones((CELL_IMG_H, CELL_IMG_W, 3))
    cell_img += BG_COLOUR
    
    
    # positioning ego in cell
    egoX = int((CELL_IMG_W - EGO_IMG_W) / 2)
    egoY = int((CELL_IMG_H - EGO_IMG_H) / 2)
    cell_img[egoY : egoY+EGO_IMG_H, egoX:egoX+EGO_IMG_W] = ego_img
    
    if debug :
        return cell_img, has_issue_lateral, has_issue_longitudinal
    else :
        return cell_img, has_issue_lateral or has_issue_longitudinal
    

In [ ]:
def make_target_img_cell(i, target_idx, target_lateral, target_longitudinal, target_velocity, debug=False) :
    if debug :
        print(i, target_idx, target_lateral, target_longitudinal, target_pos_lateral, target_pos_longitudinal, target_velocity)
      
    cell_img = np.ones((CELL_IMG_H, CELL_IMG_W, 3))
    cell_img += BG_COLOUR
    
    # is there a target at all?
    if np.sum(np.array([target_lateral, target_longitudinal, target_velocity]) == '') == 3 :
        # all five features are '' => no target => return empty cell
        return False, cell_img, False
    
    # we have an object!
    target_img = np.zeros((TARGET_IMG_H, TARGET_IMG_W, 3))
    has_issue_longitudinal = False
    has_issue_lateral = False
    has_issue_velocity = False
   
    # processing longitudinal information
    if target_longitudinal == 'a' :
        target_img += TARGET_ACCELERATING_COLOUR
    elif target_longitudinal == 'd' :
        target_img += TARGET_DECELERATING_COLOUR
    elif target_longitudinal == 'c' :
        target_img += TARGET_CRUISING_COLOUR
    else :
        target_img += TARGET_ISSUE
        has_issue_longitudinal = True
        
    # processing lateral information
    if target_lateral in ['ro', 'ri'] :
        # we place the blinker at the front right
        x0 = TARGET_IMG_W - BLINKER_W
        y0 = TARGET_IMG_H - BLINKER_H
        target_img[y0:y0+BLINKER_H, x0:x0+BLINKER_W] = TARGET_BLINKER_COLOUR
    elif target_lateral in ['lo', 'li'] :
        # we place the blinker at the front left
        x0 = TARGET_IMG_W - BLINKER_W
        y0 = 0
        target_img[y0:y0+BLINKER_H, x0:x0+BLINKER_W] = TARGET_BLINKER_COLOUR
    elif target_lateral == 'fl' :
        # lane following => no blinker
        pass
    else :
        target_img += TARGET_ISSUE
        has_issue_lateral = True
        
    # processing velocity information
    targetX = None
    targetY = int((CELL_IMG_H - TARGET_IMG_H) / 2)
    if target_velocity == 'equal' :
        # same speed => centred
        targetX = int((CELL_IMG_W - TARGET_IMG_W) / 2)
    elif target_velocity == 'slower' :
        # draw the "slower than" lines
        # draw "skidmarks" at the very right
        targetX = 2
        for x in range(CELL_IMG_W - TARGET_IMG_W - 2 + 1, CELL_IMG_W - 2 + 1, 2) :
            for y in range(targetY, targetY + TARGET_IMG_H, 2) :
                cell_img[y:y+1, x:x+1] = TARGET_REL_SPEED_COLOUR   
    elif target_velocity == 'faster' :
        # at the very left
        targetX = CELL_IMG_W - TARGET_IMG_W - 2
        # draw the "faster than" lines
        for x in range(2, CELL_IMG_W - TARGET_IMG_W) :
            for y in range(targetY, targetY + TARGET_IMG_H, 2) :
                cell_img[y:y+1, x:x+1] = TARGET_REL_SPEED_COLOUR
    else :
        # unknown: centre image + issue
        targetX = int((CELL_IMG_W - TARGET_IMG_W) / 2)
        has_issue_velocity = True
        
    # pasting target in cell
    cell_img[targetY : targetY + TARGET_IMG_H, targetX : targetX + TARGET_IMG_W] = target_img
    
    if debug :
        return True, cell_img, has_issue_lateral, has_issue_longitudinal, has_issue_velocity
    else :
        return True, cell_img, has_issue_lateral or has_issue_longitudinal or has_issue_velocity
    

In [ ]:
def make_macro_cell_EGO(i, ego_img) :
    # this one is simple
    macro_cell_img = np.zeros((MACRO_CELL_H, MACRO_CELL_W, 3))
    macro_cell_img += BG_COLOUR
    
    egoX = int((MACRO_CELL_W - CELL_IMG_W) / 2)
    egoY = int((MACRO_CELL_H - CELL_IMG_H) / 2)
    
    macro_cell_img[egoY:egoY+CELL_IMG_H, egoX:egoX+CELL_IMG_W] = ego_img
    
    return macro_cell_img    

In [ ]:
def make_macro_cell_target(i, lateral_position, longitudinal_position, target_img_list, target_position) :
    
    macro_cell_img = np.zeros((MACRO_CELL_H, MACRO_CELL_W, 3))
    macro_cell_img += BG_COLOUR 
    
    for t in range(nTargets) :
        target_img = target_img_list[t]
        target_pos = target_position[t]
        
        if target_pos['lateral'] == lateral_position and target_pos['longitudinal'] == longitudinal_position :
            # this target can be added to the macro cell
            #print(t)
            targetX = int(int(t / 4) * MACRO_CELL_W / 2)
            targetY = int(t % 4 * MACRO_CELL_H / 4)
            macro_cell_img[targetY : targetY + CELL_IMG_H, targetX : targetX + CELL_IMG_W] = target_img
        
    return macro_cell_img

In [ ]:
def make_full_img(i, ego_img, target_img_list, target_rel_pos) :
    
    # initialising full image
    n_rows = len(valid_relative_positions['lateral']) - 1 # -1 because 'lateral' also contains ''
    n_cols = len(valid_relative_positions['longitudinal']) # so basically all longitudinal + EGO CELL which it is always at centre
    full_img = np.zeros((n_rows * MACRO_CELL_H + (n_rows - 1) * H_LANE_H, 
                         n_cols * MACRO_CELL_W + (n_cols - 1) * V_LANE_W, 3))
    full_img += BG_COLOUR
    
    #print(n_rows, n_cols)
    
    # adding horizontal lanes
    for i in range(1, n_rows) :
        y0 = MACRO_CELL_H * i + (i-1) * H_LANE_H
        for x in range(0, full_img.shape[1], 2 * H_LANE_W) :
            full_img[y0 : y0 + H_LANE_H, x : x + H_LANE_W] = H_LANE_COLOUR
            
    # adding vertical lanes
    for j in range(1, n_cols) :
        x0 = MACRO_CELL_W * j + (j-1) * V_LANE_W
        for y in range(0, full_img.shape[0], 2 * V_LANE_H) :
            full_img[y : y + V_LANE_H, x0 : x0 + V_LANE_W] = V_LANE_COLOUR
            
    # ego img positioning: always in the centre
    ego_col = int(n_cols / 2)
    x0 = ego_col * MACRO_CELL_W + int((n_cols - 1) / 2) * V_LANE_W
    y0 = int(n_rows / 2) * MACRO_CELL_H + int((n_rows - 1) / 2) * H_LANE_H
    macro_cell_ego = make_macro_cell_EGO(i, ego_img)
    full_img[y0 : y0 + MACRO_CELL_H, x0 : x0 + MACRO_CELL_W] = macro_cell_ego
    
    for lat_i in range(len(valid_relative_positions['lateral'])) :
        lat = valid_relative_positions['lateral'][lat_i]
        if lat == '' :
            continue
        y0 = MACRO_CELL_H * lat_i
        if lat_i > 0 :
            y0 += lat_i * H_LANE_H
        
        x0 = None
        for lon_i in range(len(valid_relative_positions['longitudinal'])) :
            lon = valid_relative_positions['longitudinal'][lon_i]
            if lon == '' :
                continue
            x0 = lon_i * MACRO_CELL_W
            if lon_i > 0 :
                x0 += lon_i * V_LANE_W
            if lon_i >= ego_col :
                x0 += MACRO_CELL_W + V_LANE_W

            macro_cell_target = make_macro_cell_target(i, lat, lon, target_img_list, target_position)
            full_img[y0 : y0 + MACRO_CELL_H, x0 : x0 + MACRO_CELL_W] = macro_cell_target

    return full_img

# GENERAL SETTINGS

In [ ]:
projectName

In [ ]:
print_valid_relative_positions(valid_relative_positions)

In [ ]:
baseFolder = os.path.join('..', 'data')

In [ ]:
inputFolder = "00_raw_data"

In [ ]:
outputFolder = "01_activity_dictionary"

In [ ]:
img_outputSubFolder = "img_{:d}x{:d}".format(len(valid_relative_positions['lateral'])-1,
                                             len(valid_relative_positions['longitudinal'])-1)

In [ ]:
dictionaryFilename = "activityDictionary_{:d}x{:d}.csv".format(len(valid_relative_positions['lateral'])-1,
                                                               len(valid_relative_positions['longitudinal'])-1)

In [ ]:
dictionaryFilename

In [ ]:
inputPath = os.path.join(baseFolder, inputFolder)

In [ ]:
inputPath_dictionary = os.path.join(baseFolder, outputFolder)
outputPath_dictionary = os.path.join(baseFolder, outputFolder)
outputPath_dictionary

In [ ]:
outputPath_dictionary_img = os.path.join(outputPath_dictionary, img_outputSubFolder)

In [ ]:
outputPath_dictionary_img

In [ ]:
if outputFolder not in os.listdir(baseFolder) :
    print("creating", outputPath_dictionary)
    os.mkdir(outputPath_dictionary)

In [ ]:
if img_outputSubFolder not in os.listdir(outputPath_dictionary) :
    print("creating", outputPath_dictionary_img)
    os.mkdir(outputPath_dictionary_img)

# LOAD TAGGED DATASET

In [ ]:
fileList = os.listdir(inputPath)

In [ ]:
fileList

In [ ]:
inputFile = "20170529_PP_03_Run_1.hdf5"

In [ ]:
inputPath + inputFile

In [ ]:
s = pd.HDFStore(inputPath + inputFile)
tagged_dataset = s.get('df')
s.close()

In [ ]:
tagged_dataset.shape

In [ ]:
tagged_dataset.reset_index(inplace=True, drop=True)

In [ ]:
tagged_dataset.head()

In [ ]:
valuesPerCol = {}
colsWithRawValues = []
lastTargetID = 0
for c in tagged_dataset.columns :
    if tagged_dataset[c].dtype == float :
        colsWithRawValues.append(c)
        continue
    cName = c
    if 'host' in c[:len('host')] :
        if c not in valuesPerCol :
            valuesPerCol[c] = []
    elif 'target' in c[:len('target')] :
        targetId = int(c.split('_')[1])
        if targetId > lastTargetID :
            lastTargetID = targetId
        last = '_'.join(c.split('_')[2:])
        cName = 'target_' + last
        if cName not in valuesPerCol :
            valuesPerCol[cName] = []
    uniq = list(tagged_dataset[c].drop_duplicates())
    valuesPerCol[cName] = list(set(valuesPerCol[cName] + uniq))
print("categorical columns and values:")
for k in valuesPerCol :
    print("\t", k, valuesPerCol[k])
print("\ncontinuous columns:")
for k in colsWithRawValues :
    print("\t", k)

In [ ]:
nTargets = lastTargetID + 1

In [ ]:
nTargets

# INITIALISE DICTIONARY
either load existing one or create an empty one

In [ ]:
dictionary = None
if dictionaryFilename in os.listdir(inputPath_dictionary) :
    print("loading", outputPath_dictionary + dictionaryFilename)
    dictionary = pd.read_pickle(outputPath_dictionary + dictionaryFilename)
else :
    dictionary = pd.DataFrame(columns=dictionary_columns)  #['index', 'host_lateral', 'host_longitudinal', 'target_0_lateral', 'target_0_longitudinal', 'target_0_relative_position_longitudinal', 'target_0_relative_position_lateral', 'target_0_velocity', 'target_1_lateral', 'target_1_longitudinal', 'target_1_relative_position_longitudinal', 'target_1_relative_position_lateral', 'target_1_velocity', 'target_2_lateral', 'target_2_longitudinal', 'target_2_relative_position_longitudinal', 'target_2_relative_position_lateral', 'target_2_velocity', 'target_3_lateral', 'target_3_longitudinal', 'target_3_relative_position_longitudinal', 'target_3_relative_position_lateral', 'target_3_velocity', 'target_4_lateral', 'target_4_longitudinal', 'target_4_relative_position_longitudinal', 'target_4_relative_position_lateral', 'target_4_velocity', 'target_5_lateral', 'target_5_longitudinal', 'target_5_relative_position_longitudinal', 'target_5_relative_position_lateral', 'target_5_velocity', 'target_6_lateral', 'target_6_longitudinal', 'target_6_relative_position_longitudinal', 'target_6_relative_position_lateral', 'target_6_velocity', 'target_7_lateral', 'target_7_longitudinal', 'target_7_relative_position_longitudinal', 'target_7_relative_position_lateral', 'target_7_velocity', 'n_objects',  'ego_img',  'target_0_img',  'target_1_img',  'target_2_img',  'target_3_img',  'target_4_img',  'target_5_img', 'target_6_img',  'target_7_img',  'full_img',  'ego_issue', 'target_0_issue', 'target_1_issue', 'target_2_issue', 'target_3_issue', 'target_4_issue', 'target_5_issue', 'target_6_issue', 'target_7_issue' ])        

In [ ]:
dictionary.shape

In [ ]:
dictionary.columns == dictionary_columns

# TAGGED DATAFRAME CORRECTION BASED ON VALID RELATIVE POSITIONS

In [ ]:
reduced_dataset = tagged_dataset.copy()

In [ ]:
print(dictionary.shape, len(core_dictionary_columns), len(img_dictionary_columns))
print(tagged_dataset[core_dictionary_columns].shape)
print(tagged_dataset[core_dictionary_columns].drop_duplicates().shape)

In [ ]:
for t in range(nTargets) :
    reduced_dataset.loc[(~reduced_dataset['target_' + str(t) + '_relative_position_longitudinal'].isin(valid_relative_positions['longitudinal'])) |
              (~reduced_dataset['target_' + str(t) + '_relative_position_lateral'].isin(valid_relative_positions['lateral'])),
                        ['target_' + str(t) + '_lateral', 
                         'target_' + str(t) + '_longitudinal',
                         'target_' + str(t) + '_relative_position_longitudinal', 
                         'target_' + str(t) + '_relative_position_lateral',
                         'target_' + str(t) + '_velocity']] = ''
    #break

In [ ]:
for t in range(8) :
    print(t, list(reduced_dataset['target_' + str(t) + '_relative_position_lateral'].drop_duplicates()))

# DICTIONARY CONSTRUCTION OF THE CORRECTED DATAFRAME
so basically we check which entries are new wrt the existing dictionary

In [ ]:
print(reduced_dataset[core_dictionary_columns].drop_duplicates().shape)

In [ ]:
unique_entries = reduced_dataset[core_dictionary_columns].drop_duplicates()

In [ ]:
unique_entries.reset_index(inplace=True, drop=True)

In [ ]:
unique_entries.shape

In [ ]:
for c in unique_entries.columns :
    if c not in dictionary.columns :
        print(c)
print('.')

In [ ]:
for c in dictionary.columns :
    if c not in unique_entries.columns :
        print(c)

In [ ]:
n_add = 0
n_skip = 0

for i in range(len(unique_entries)) :
    print(i, len(unique_entries), 10*' ', end='\r')
    
    # is the entry already present in the dictionary?
    entry = unique_entries.iloc[i]
    
    add_entry = False
    if len(dictionary) == 0 :
        add_entry = True
    elif np.max(np.sum(dictionary[entry.index] == entry, axis=1)) < len(entry) :
        # the highest number of common features is less than the lenght of the entry => new feature values => add!
        add_entry = True
        
    if add_entry :        
        # we can add this entry
        entry['index'] = len(dictionary)
        entry['n_objects'] = sum([entry['target_' + str(t) + "_lateral"] != '' for t in range(nTargets)])
        entry['ego_img'], entry['ego_issue'] = make_EGO_IMG_cell(i, entry['host_lateral'], entry['host_longitudinal'])
        
        n_targets = []
        target_position = []
        
        for t in range(nTargets) :
            target_lateral = entry['target_' + str(t) + '_lateral']
            target_longitudinal = entry['target_' + str(t) + '_longitudinal']
            target_velocity = entry['target_' + str(t) + '_velocity']
            target_exists, entry['target_' + str(t) + '_img'], entry['target_' + str(t) + '_issue'] = make_target_img_cell(i, t, target_lateral, target_longitudinal, target_velocity)
            n_targets.append(target_exists)
            target_position.append({'lateral' : entry['target_' + str(t) + '_relative_position_lateral'],
                                   'longitudinal' : entry['target_' + str(t) + '_relative_position_longitudinal']})
            #break
        
        # time to merge the EGO/target images into a macro image
        entry['full_img'] = make_full_img(i, entry['ego_img'], [entry['target_' + str(x) + '_img'] for x in range(nTargets)], target_position)
        
        # sanity check: do we have all entries to add to the dictionarY?
        all_good = sum([c in dictionary.columns for c in entry.index]) == len(dictionary.columns)
        if all_good :
            #print("all good to add!")
            n_add += 1
            dictionary = dictionary.append(entry, ignore_index=True)
            
    else :
        #print("already present")
        n_skip += 1
    
print('\n.')

In [ ]:
entry

In [ ]:
render(entry['target_7_img'])

In [ ]:
print("dataset processed, new entries =", n_add, "existing entries =", n_skip)
print("updated dictionary shape=", dictionary.shape)
print("issues status:")
for c in dictionary.columns :
    if 'issue' in c :
        print(c, sum(dictionary[c]))

# DICTIONARY FINALISATION
i.e. saving the dictionary and its images

In [ ]:
dictionary.to_pickle(outputPath_dictionary + dictionaryFilename)

In [ ]:
for i in range(len(dictionary)) :
    print(i, len(dictionary), 10*' ', end='\r')
    render(dictionary.iloc[i]['full_img'], saveFig=True, title=str(i), outPath=outputPath_dictionary_img)
print('\n.')